In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word2idx = {"hello": 0, "world": 1}
embed = nn.Embedding(2, 5) # shape (2, 5)
hello_lookup_tensor = torch.LongTensor([word2idx["hello"]])
hello_embed = embed(autograd.Variable(hello_lookup_tensor))
print(hello_embed)


Variable containing:
 0.8407  0.5510  0.3863  0.9124 -0.8410
[torch.FloatTensor of size 1x5]



In [20]:
### NgramLanguageModel

context_size = 2
embedding_dim = 10

test_sent = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# trigrams: list[tuple(list[context], target), ...] [([context], target), ...]
trigrams = [([test_sent[i-2], test_sent[i-1]], test_sent[i]) for i in range(2, len(test_sent))]
print(trigrams[:3])

vocab = set(test_sent)
vocab_size = len(vocab) # 97
word2idx = {word: i for i, word in enumerate(vocab)}

class NGramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim) # embeds inputs from R^vocab_size to R^embed_dim
        self.linear1 = nn.Linear(context_size * embedding_dim, 128) # maps embeddings from R^ctx_size*embed to R^128
        self.linear2 = nn.Linear(128, vocab_size) # maps linear1_out from R^128 to R^vocab_size(97)
    
    def forward(self, inputs):
        '''inputs: vector of two context words' index, shape (2) '''
        embedding = self.embedding(inputs).view(1, -1) # shape (1, 20) ctx_size(2)*embed_dim(10)=20
        linear1_out = F.relu(self.linear1(embedding)) # shape (1, 128)
        linear2_out = self.linear2(linear1_out) # shape (1, 97) vocab_size=97
        log_probs = F.log_softmax(linear2_out, dim=1) # shape (1, 97) vocab_size=97
        return log_probs

losses = []
loss_fun = nn.NLLLoss()
model = NGramLanguageModel(vocab_size, embedding_dim, context_size)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        # step1. prepare the inputs
        context_idxs = [word2idx[w] for w in context]
        context_idxs_var = autograd.Variable(torch.LongTensor(context_idxs))
        
        # step2. clear the grad
        model.zero_grad()
        
        # step3. forward and compute the loss
        log_probs = model(context_idxs_var)
        loss = loss_fun(log_probs, autograd.Variable(torch.LongTensor([word2idx[target]])))
        
        # step4. backward and update the params
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
print(losses)


[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[
 521.5227
[torch.FloatTensor of size 1]
, 
 518.9681
[torch.FloatTensor of size 1]
, 
 516.4318
[torch.FloatTensor of size 1]
, 
 513.9138
[torch.FloatTensor of size 1]
, 
 511.4140
[torch.FloatTensor of size 1]
, 
 508.9315
[torch.FloatTensor of size 1]
, 
 506.4641
[torch.FloatTensor of size 1]
, 
 504.0102
[torch.FloatTensor of size 1]
, 
 501.5681
[torch.FloatTensor of size 1]
, 
 499.1355
[torch.FloatTensor of size 1]
]


In [23]:
### CBOW word embedding
context_size = 2 # 2 words to the left, 2 to the right
embedding_dim = 10
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

vocab = set(raw_text)
vocab_size = len(vocab) # 49

word2idx = {word: i for i, word in enumerate(vocab)}
# data: list[tuple(list[context], target), ...] [([context], target), ...]
data = []
for i in range(2, len(raw_text) - 2):
    contex = [raw_text[i-2], raw_text[i-1], raw_text[i+1], raw_text[i+2]]
    data.append((context, raw_text[i]))
print(data[:5])    

class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # embeds inputs from R^vocab_size to R^embed_dim
        self.linear = nn.Linear(embedding_dim, vocab_size) # maps embedddings from R^embed_dim to R^vocab_size
    
    def forward(self, inputs):
        '''inputs: vector of four context word, shape (4)'''
        embeddings = self.embeddings(inputs) # shape (4, 10) embed_dim=10
        embeddings_mean = autograd.Variable(torch.mean(embeddings.data, dim=0, keepdim=True)) # shape (1, 10)
        linear_out = self.linear(embeddings_mean) # shape (1, 49) vocab_size=49
        log_probs = F.log_softmax(linear_out, dim=1) # shape (1, 49)
        return log_probs
    
def make_contex_vec(context, word2idx):
    context_idxs = [word2idx[w] for w in contex]
    context_tensor = torch.LongTensor(context_idxs)
    return autograd.Variable(context_tensor)
    
#make_contex_vec(data[0][0], word2idx)  

losses = []
loss_fun = nn.NLLLoss()
cbow_model = CBOW(vocab_size, embedding_dim, context_size)
optimizer = optim.SGD(cbow_model.parameters(), lr=0.01)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in data:
        # step1, prepare the inputs
        context_var = make_contex_vec(context, word2idx)
        
        # step2, clear the grad
        cbow_model.zero_grad()
        
        # step3, forward and compute the loss
        log_probs = cbow_model(context_var)
        loss = loss_fun(log_probs, autograd.Variable(torch.LongTensor(word2idx[target])))
        
        # step4, backward and update the params
        loss.backward()
        optimizer.step()
    
        total_loss += loss.data
    losses.append(total_loss)
print(losses)


[(["feel'st", 'it'], 'about'), (["feel'st", 'it'], 'to'), (["feel'st", 'it'], 'study'), (["feel'st", 'it'], 'the'), (["feel'st", 'it'], 'idea')]


RuntimeError: Assertion `THIndexTensor_(size)(target, 0) == batch_size' failed.  at /opt/conda/conda-bld/pytorch_1513368888240/work/torch/lib/THNN/generic/ClassNLLCriterion.c:79